### Imports

In [ ]:
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import numpy as np
import pandas as pd
from datetime import datetime

import backtrader as bt
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

In [ ]:
import sys

sys.path.append('./Strategies')
sys.path.append('./DataFeeds')

from glove import *
#from buy_and_hold_strats import *
from random_strats import *
from mean_ou_strats import *
from high_mid_low_strats import *

### Strategy Testing

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLow(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
    
    def update_dataset(self, order, close = False):
        if not close:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            

            if not self.position:
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order, close = True)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            if self.datas[0].sentiment_weighted_mean >= 0.00:
            # Mid
#             if self.datas[0].sentiment_mean < 0.7 and self.datas[0].sentiment_mean > 0.633:
            # Low
#             if self.datas[0].sentiment_mean <= 0.633:
                #print(self.broker.getvalue(), self.broker.getcash())
                
        
                #position sizing
                #pos_size = self.broker_val_diff()
#                 pos_size = self.single()
                     
#              
                if self.position.size:
                    self.close(exectype=bt.Order.Market, coc = True)
#                 pos_size = abs(self.datas[0].sentiment_weighted_mean)/self.datas[0].open[0]
                #stop loss price
                sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
                #take profit price
                tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#             if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)

                #Bracket order buy
                self.order = self.buy_bracket(exectype=bt.Order.Market, 
                                     stopprice = sl_price,
                                     limitprice = tp_price,
                                     coc=False)

#                 new_pos = pos_size - self.position.size
#                 if new_pos > 0:
#                     self.order = self.buy(exectype=bt.Order.Market, size = abs(new_pos))
#                 elif new_pos < 0:
#                     self.order = self.sell(exectype=bt.Order.Market, size = abs(new_pos))
        #Close all positions at the Close of the day
            
        if self.position.size:
            #self.log(str(len(self))+' '+str(self.size))
            if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
            #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
                self.order = self.close(exectype=bt.Order.Market, coc = True)
                # and Cancel all pending orders
                for o in self.p.orders:
                    order = self.p.orders[o]
                    if order.alive :
                        self.cancel(order)
                self.p.orders = dict()
                
                if len(self) == self.size:
                     self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
                     self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLow(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
    
    def update_dataset(self, order, close = False):
        if not close:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            

            if not self.position:
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order, close = True)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
                sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
                #take profit price
                tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)

                #Bracket order buy
                self.order = self.buy_bracket(exectype=bt.Order.Market, 
                                     stopprice = sl_price,
                                     limitprice = tp_price,
                                     coc=False)
               
            
        if self.position.size:
            #self.log(str(len(self))+' '+str(self.size))
            if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
            #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
                self.order = self.close(exectype=bt.Order.Market, coc = True)
                # and Cancel all pending orders
                for o in self.p.orders:
                    order = self.p.orders[o]
                    if order.alive :
                        self.cancel(order)
                self.p.orders = dict()
                
                if len(self) == self.size:
                     self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
                     self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

### Running Backtest

In [ ]:
#x = pd.read_csv('../Data Preprocessed/glove_1M_full_dates_hour.csv')
x = pd.read_csv('../Data Preprocessed/glove_month_fin.csv')
x['Bitcoin'] = x['sentiment']
x.info(verbose = True)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
high = x['Bitcoin'].max()
low = x['Bitcoin'].min()
x['btc_norm'] =  x['Bitcoin'].apply(lambda x: (x - low)/ (high - low))

high = x['Open'].max()
low = x['Open'].min()
x['open_norm'] =  x['Open'].apply(lambda x: (x - low)/ (high - low))
x

In [ ]:
l = x.loc[1:]['Open']

In [ ]:
l = l.reset_index(drop=True)
l

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

#ax.bar(x['day'],x['ret'],1, label='sentiment')
ax.plot(x['btc_norm'].values[:-1])
ax.plot(x['open_norm'].values[1:], alpha = 0.5)
#ax.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
#plt.plot([0, 1], [0, 1], 'k--')
ax.set_title('Precision - Recall curve', fontsize=20)
ax.set_ylabel('Sentiment', fontsize=16)
ax.set_xlabel('month-year', fontsize=16)
#ax.set_ylim(0.5,1)
leg = ax.legend(loc='best')
#plt.savefig('comparison_prerecall.png', facecolor='w')
plt.show()

In [ ]:
import statsmodels.api as sm
#Y = pd.DataFrame(l.values).astype(float)
#Y = dataset_standarization(Y)
Y = pd.DataFrame(x['open_norm'].values[1:]).astype(float)
#Textual data ('Bitcoin' topic popularity) of the last hour standarized
X = pd.DataFrame(x['btc_norm'].values[:-1]).astype(float)
#X = dataset_standarization(X)
Y.index = X.index

X = sm.add_constant(X)

model = sm.OLS(Y, X, missing = 'drop').fit()
print(model.summary())

In [ ]:
# x.loc[x['Bitcoin'].isna(),'Bitcoin'] = 7
x

In [ ]:
x.describe()

In [ ]:
np.quantile(x['Bitcoin'].dropna().values, 0.04)

### Strategies

#### HighMidLow

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLow(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
    
    def update_dataset(self, order, close = False):
        if not close:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            

            if not self.position:
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order, close = True)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
            lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]  and not self.position.size:
                sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
                #take profit price
                tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                self.buy(exectype=bt.Order.Market)
            elif self.datas[0].sentiment_mean[0] >= self.datas[0].sentiment_mean[-1] and not self.position.size:
                self.sell(exectype=bt.Order.Market)
                #Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
               
            
        if self.position.size:
            #self.log(str(len(self))+' '+str(self.size))
            if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
            #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
                self.order = self.close(exectype=bt.Order.Market, coc = True)
                # and Cancel all pending orders
                for o in self.p.orders:
                    order = self.p.orders[o]
                    if order.alive :
                        self.cancel(order)
                self.p.orders = dict()
                
                if len(self) == self.size:
                     self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
                     self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### HighMidLowTryXma

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowTryXma(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value']),
        th = None
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
        self.cl = False
        self.threshold = self.params.th
        
    def update_dataset(self, order):
        if not self.cl:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.cl = False
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.cl  = False
            if not self.position:
                self.cl  = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
#         print(self.position)
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             self.order = self.close(exectype=bt.Order.Market, coc = True)


#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0 and self.datas[0].ma[0]!=0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
#             lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].rounded_sum[0] > self.threshold*self.datas[0].ma[0]:
                if not self.position:
#                     print(1)
                    self.cl = False
                    self.buy(exectype=bt.Order.Market, coc = False)
#                 elif self.position.size<0:
# #                     print(2)
#                     self.cl = True
#                     self.order = self.close(exectype=bt.Order.Market, coc = False)
#                     #self.sell(exectype=bt.Order.Market)
#                 sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
#                 #take profit price
#                 tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                
#             elif self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]:
#                 if not self.position:
# #                     print(3)
#                     self.cl = False
# #                     self.sell(exectype=bt.Order.Market)
            elif self.position.size>0:
#                     print(4)
                self.cl = True
                self.order = self.close(exectype=bt.Order.Market, coc = False)
                #self.buy(exectype=bt.Order.Market)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
        elif self.position:
            self.cl = True
            self.order = self.close(exectype=bt.Order.Market, coc = True)
            
        if self.position.size:
            #self.log(str(len(self))+' '+str(self.size))
            if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
            #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
                self.order = self.close(exectype=bt.Order.Market, coc = True)
                # and Cancel all pending orders
                for o in self.p.orders:
                    order = self.p.orders[o]
                    if order.alive :
                        self.cancel(order)
                self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### HighMidLowTryXmaMUL HOUR

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowTryXmamulh(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value']),
        th = None,
        p=0
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
        self.cl = False
        self.threshold = self.params.th
        self.posnum = self.params.p
        self.psize = 0
        
    def update_dataset(self, order):
        if not self.cl:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.cl = False
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.cl  = False
            if not self.position:
                self.cl  = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        
        if self.position.size>0:
            #self.log(str(len(self))+' '+str(self.size))
            if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
            #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
                print("END DAY")
                self.order = self.close(exectype=bt.Order.Market, coc = True)
#         print(self.position)
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             self.order = self.close(exectype=bt.Order.Market, coc = True)


#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if not self.position:
            self.psize = self.broker.getvalue()*0.95/self.posnum
            
        if self.datas[0].sentiment_mean > -2.0 and self.datas[0].ma[0]!=0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
#             lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].rounded_sum[0] > self.threshold*self.datas[0].ma[0]:
#             if not self.position:
    #                     print(1)
                #self.cl = False
                if len(self) != self.size:
                    self.buy(exectype=bt.Order.Market, coc = False, size = self.psize/self.datas[0].open[1])
#                     print(len(self.datas[0].open))
                else:
                    print("END")
                    self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 elif self.position.size<0:
# #                     print(2)
#                     self.cl = True
#                     self.order = self.close(exectype=bt.Order.Market, coc = False)
#                     #self.sell(exectype=bt.Order.Market)
#                 sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
#                 #take profit price
#                 tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                
#             elif self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]:
#                 if not self.position:
# #                     print(3)
#                     self.cl = False
# #                     self.sell(exectype=bt.Order.Market)
            elif self.position.size>0:
#                     print(4)
                self.cl = True
                self.order = self.close(exectype=bt.Order.Market, coc = False)
                #self.buy(exectype=bt.Order.Market)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
        elif self.position:
            self.cl = True
            self.order = self.close(exectype=bt.Order.Market, coc = True)
            

#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### HighMidLowTryXmamul

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowTryXmamul(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value']),
        th = None,
        p=0
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
        self.cl = False
        self.threshold = self.params.th
        self.posnum = self.params.p
        self.psize = 0
        
    def update_dataset(self, order):
        if not self.cl:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.cl = False
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.cl  = False
            if not self.position:
                self.cl  = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
#         print(self.position)
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             self.order = self.close(exectype=bt.Order.Market, coc = True)


#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if not self.position:
            self.psize = self.broker.getvalue()*0.95/self.posnum
        #print(self.datas[0].rounded_sum[0], self.datas[0].ma[0])
        if self.datas[0].sentiment_mean[0] > -2.0 and self.datas[0].ma[0]!=0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
#             lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].rounded_sum[0] > self.threshold*self.datas[0].ma[0]:
#             if not self.position:
    #                     print(1)
                #self.cl = False
                if len(self) != self.size:
                    self.buy(exectype=bt.Order.Market, coc = False, size = self.psize/self.datas[0].open[1])
#                     print(len(self.datas[0].open))
                else:
                    print("END")
                    self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 elif self.position.size<0:
# #                     print(2)
#                     self.cl = True
#                     self.order = self.close(exectype=bt.Order.Market, coc = False)
#                     #self.sell(exectype=bt.Order.Market)
#                 sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
#                 #take profit price
#                 tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                
#             elif self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]:
#                 if not self.position:
# #                     print(3)
#                     self.cl = False
# #                     self.sell(exectype=bt.Order.Market)
            elif self.position.size>0:
#                     print(4)
                self.cl = True
                self.order = self.close(exectype=bt.Order.Market, coc = False)
                #self.buy(exectype=bt.Order.Market)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
        elif self.position:
            self.cl = True
            self.order = self.close(exectype=bt.Order.Market, coc = True)
            
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
#             #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
#                 self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowTryXma(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value']),
        th = None
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
        self.cl = False
        self.threshold = self.params.th
        
    def update_dataset(self, order):
        if not self.cl:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.cl = False
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.cl  = False
            if not self.position:
                self.cl  = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
#         print(self.position)
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             self.order = self.close(exectype=bt.Order.Market, coc = True)


#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0 and self.datas[0].ma[0]!=0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
#             lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].rounded_sum[0] > self.threshold*self.datas[0].ma[0]:
                if not self.position:
#                     print(1)
                    self.cl = False
                    self.buy(exectype=bt.Order.Market, coc = False)
#                 elif self.position.size<0:
# #                     print(2)
#                     self.cl = True
#                     self.order = self.close(exectype=bt.Order.Market, coc = False)
#                     #self.sell(exectype=bt.Order.Market)
#                 sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
#                 #take profit price
#                 tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                
#             elif self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]:
#                 if not self.position:
# #                     print(3)
#                     self.cl = False
# #                     self.sell(exectype=bt.Order.Market)
            elif self.position.size>0:
#                     print(4)
                self.cl = True
                self.order = self.close(exectype=bt.Order.Market, coc = False)
                #self.buy(exectype=bt.Order.Market)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
        elif self.position:
            self.cl = True
            self.order = self.close(exectype=bt.Order.Market, coc = True)
            
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
#             #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
#                 self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### HighMidLowTryX

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowTryX(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
        self.cl = False
        self.threshold = 0.98
        
    def update_dataset(self, order):
        if not self.cl:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.cl = False
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.cl  = False
            if not self.position:
                self.cl  = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
#         print(self.position)
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             self.order = self.close(exectype=bt.Order.Market, coc = True)


#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
#             lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            
            if self.datas[0].sentiment_mean[0] > self.threshold*self.datas[0].sentiment_mean[-1]:
                if not self.position:
#                     print(1)
                    self.cl = False
                    self.buy(exectype=bt.Order.Market, coc = False)
#                 elif self.position.size<0:
# #                     print(2)
#                     self.cl = True
#                     self.order = self.close(exectype=bt.Order.Market, coc = False)
#                     #self.sell(exectype=bt.Order.Market)
#                 sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
#                 #take profit price
#                 tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                
#             elif self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]:
#                 if not self.position:
# #                     print(3)
#                     self.cl = False
# #                     self.sell(exectype=bt.Order.Market)
            elif self.position.size>0:
#                     print(4)
                self.cl = True
                self.order = self.close(exectype=bt.Order.Market, coc = False)
                #self.buy(exectype=bt.Order.Market)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
        elif self.position:
            self.cl = True
            self.order = self.close(exectype=bt.Order.Market, coc = True)
            
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
#             #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
#                 self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### HighMidLowTryX2

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowTryX2(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
        self.cl = False
        
    def update_dataset(self, order):
        if not self.cl:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.cl = False
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.cl  = False
            if not self.position:
                self.cl  = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
#         print(self.position)
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             self.order = self.close(exectype=bt.Order.Market, coc = True)


#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
#             lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].sentiment_mean[0] >= self.datas[0].sentiment_mean[-1]:
                if not self.position:
#                     print(1)
                    self.cl = False
                    self.buy(exectype=bt.Order.Market, coc = False)
                elif self.position.size<0:
#                     print(2)
                    self.cl = True
                    self.order = self.close(exectype=bt.Order.Market, coc = False)
                    #self.sell(exectype=bt.Order.Market)
#                 sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
#                 #take profit price
#                 tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                
            elif self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]:
                if not self.position:
#                     print(3)
                    self.cl = False
                    self.sell(exectype=bt.Order.Market, coc = False)
                elif self.position.size>0:
#                     print(4)
                    self.cl = True
                    self.order = self.close(exectype=bt.Order.Market, coc = False)
#                     self.buy(exectype=bt.Order.Market)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
        elif self.position:
            self.cl = True
            self.order = self.close(exectype=bt.Order.Market, coc = True)
            
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
#             #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
#                 self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### HighMidLowBest

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowBest(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
    
    def update_dataset(self, order, close = False):
        if not close:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            
            close = False
            if not self.position:
                close = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order,close)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order,close)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        
        if self.position.size:
            #self.log(str(len(self))+' '+str(self.size))
            self.order = self.close(exectype=bt.Order.Market, coc = True)
#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0:
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
            lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].sentiment_mean[0] >= self.datas[0].sentiment_mean[-1]*1.001  and not self.position.size:
                sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
                #take profit price
                tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                self.sell(exectype=bt.Order.Market)
            elif self.datas[0].sentiment_mean[0] < self.datas[0].sentiment_mean[-1]/1.001 and not self.position.size:
                self.buy(exectype=bt.Order.Market)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
               
            
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
#             #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
#                 self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### HighMidLowTry

In [ ]:
#Bracket Orders BUY + Cloase each day
class HighMidLowTry(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
    
    def update_dataset(self, order, close = False):
        if not close:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            
            close = False
            if not self.position:
                close = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order,close)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order,close)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        
        if self.position.size:
            #self.log(str(len(self))+' '+str(self.size))
            self.order = self.close(exectype=bt.Order.Market, coc = True)
#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data
        if self.datas[0].sentiment_mean > -2.0:
            
            # High
#             if self.datas[0].sentiment_mean >= self.datas[0].rolling_mean:
            #if self.datas[0].sentiment_weighted_mean <= 0.0 and not self.position.size:
            lol = np.random.random()
            #if lol>0.5 and not self.position.size:
#             if self.datas[0].sentiment_mean < 0.493 and not self.position.size:
            #print(self.datas[0].rounded_sum[0],self.datas[0].rounded_sum[-1])
            if self.datas[0].strong_rounded_sum[0] >= self.datas[0].strong_rounded_sum[-1]*1.001  and not self.position.size:
                pos_size = 1 - self.datas[0].strong_rounded_sum[-1]/self.datas[0].strong_rounded_sum[0]
                pos_size *= self.broker.getvalue()
                pos_size /= self.datas[0].open[0]
                self.buy(exectype=bt.Order.Market, size = pos_size)
                sl_price = self.datas[0].open[0]*(1-self.p.stop_loss)
                #take profit price
                tp_price = self.datas[0].open[0]*(1+self.p.take_profit)
#               if pos_size>0:
#                     #Debug message
#                     self.p.msg = 'BUY CREATED AT ' + str(self.datas[0].open[0]) +' STOPLOSS SET AT ' + str(sl_price)+ '$ TAKE PROFIT AT '+ str(tp_price)+'$'
#                     #self.buy(exectype=bt.Order.Market, size = pos_size)
                #self.sell(exectype=bt.Order.Market)
            elif self.datas[0].strong_rounded_sum[0] < self.datas[0].strong_rounded_sum[-1]/1.001 and not self.position.size:
                
                pos_size = 1 - self.datas[0].strong_rounded_sum[0]/self.datas[-1].strong_rounded_sum[0]
                pos_size *= self.broker.getvalue()
                pos_size /= self.datas[0].open[0]
                self.sell(exectype=bt.Order.Market, size = pos_size)
#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
               
            
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
#             #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
#                 self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                      self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]
                

#### BuyHold

In [ ]:
#Bracket Orders BUY + Cloase each day
class BuyHold(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
    
    def update_dataset(self, order, close = False):
        if not close:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            
            close = False
            if not self.position:
                close = True
                print('(CLOSE)', end='')
            else:
                self.log(self.p.msg)
            #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order,close)
                self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order,close)
                self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log('Not Enough Margin')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        if not self.position:
            self.order = self.buy(exectype=bt.Order.Market)
#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data

#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
               
            
        if self.position.size:
            #self.log(str(len(self))+' '+str(self.size))
            if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
            #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
                self.order = self.close(exectype=bt.Order.Market, coc = True)
                # and Cancel all pending orders
                for o in self.p.orders:
                    order = self.p.orders[o]
                    if order.alive :
                        self.cancel(order)
                self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                    

#### Random1

In [ ]:
#Bracket Orders BUY + Cloase each day
class Random1(bt.Strategy):
    
    params = dict(
        stop_loss = 0.92,  # price is 2% less than the entry point
        take_profit = 10.04,
        orders = dict(),
        msg='',
        data = pd.DataFrame(columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price']),
        val = pd.DataFrame(columns = ['datetime','value'])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(sep=' ', timespec='auto'), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.sentiment = self.datas[0].sentiment_mean
        self.size = self.data.buflen()
        self.order = None
        self.stop_order = None
        self.tp_order = None
        self.total_position = 0
    
    def update_dataset(self, order, close = False):
        if not close:
            pos = 1 if order.isbuy() else -1
            data_new_line = pd.DataFrame(
                [[
                    self.datas[0].datetime.datetime(0), 
                    pos, 
                    order.size*order.executed.price,
                    order.executed.price,
                    self.data.sentiment_mean[-1],
                    None,
                    None
                ]], 
                columns = ['entry_time', 'position', 'weight', 'entry_price', 'sentiment', 'exit_time', 'exit_price'])
            self.p.data = pd.concat([self.p.data,data_new_line], axis = 0, ignore_index=False)
        else:
            self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(-1)
            self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = order.executed.price
    
    def notify_order(self, order):
        
        # Save submitted order
        if order.status in [order.Submitted, order.Accepted]:
            self.p.orders[order.ref] = order
            return
        
        # Check if an order has been completed
        if order.status in [order.Completed]:
            
            close = False
            if not self.position:
                close = True
#                 print('(CLOSE)', end='')
#             else:
# #                 self.log(self.p.msg)
#             #print(self.p.data)
            if order.isbuy():
                self.update_dataset(order,close)
#                 self.log('BUY EXECUTED, ' + str(order.size) + 'BTC AT ' +str(order.executed.price))
            elif order.issell():
                self.update_dataset(order,close)
#                 self.log('SELL EXECUTED, ' + str(order.size) + 'BTC AT ' + str(order.executed.price))
                
#         elif order.status in [order.Canceled, order.Margin, order.Rejected]:
#             if order.status == order.Canceled:
#                 self.log('Order Canceled')
#             elif order.status == order.Margin:
#                 self.log('Not Enough Margin')
#             elif order.status == order.Rejected:
#                 self.log('Order Rejected')

        self.order = None

    def broker_val_diff(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1] - self.datas[0].rolling_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean - self.datas[0].rolling_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    #high sqn big diff
    def broker_val(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        pos_size = self.broker.getvalue()*(sentiment)/self.datas[0].open[0]
        return pos_size
    
    def pure(self):
        prev_sentiment = 0
        if len(self) > 0:
            prev_sentiment = self.datas[0].sentiment_mean[-1]
        this_sentiment = self.datas[0].sentiment_mean
        sentiment = this_sentiment - prev_sentiment
        return sentiment
    
    def single(self):
         return self.broker.getvalue()*(self.datas[0].sentiment_mean)/self.datas[0].open
            
    #no difference
    def standar(self):
        return self.broker.getvalue()*0.05/self.datas[0].open
    
    def next(self):
        val_new_line = pd.DataFrame([self.datas[0].datetime.datetime(0),self.broker.getvalue()]).T
        val_new_line.columns = ['datetime','value']
        self.p.val = pd.concat([self.p.val,val_new_line], axis = 0, ignore_index=False)
        #print(self.data.sentiment_mean[0], self.data.open[0],self.data.close[0], self.data.volume[0])

        if self.order:
            return
        x = np.random.random()
        if not self.position:

            if x>=2/3:
                self.order = self.buy(exectype=bt.Order.Market)
            elif x<1/3:
                self.order = self.sell(exectype=bt.Order.Market)
        elif x<0.5:
            self.order = self.close(exectype=bt.Order.Market, coc = False)
#             # and Cancel all pending orders
#             for o in self.p.orders:
#                 order = self.p.orders[o]
#                 if order.alive :
#                     self.cancel(order)
#             self.p.orders = dict()

#             if len(self) == self.size:
#                  self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
#                  self.p.data.loc[self.p.data['exit_price'].isna() ,'exit_price'] = self.datas[0].close[0]

        #if not self.position.size:
        # this is to play only on periods with sentiment data

#                 Bracket order buy
#                 self.order = self.buy_bracket(exectype=bt.Order.Market, 
#                                      stopprice = sl_price,
#                                      limitprice = tp_price,
#                                      coc=False)
               
            
#         if self.position.size:
#             #self.log(str(len(self))+' '+str(self.size))
#             if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d'):
#             #if len(self) == self.size or self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H')!=self.datas[0].datetime.datetime(1).strftime('%Y-%m-%d %H'):
#                 self.order = self.close(exectype=bt.Order.Market, coc = True)
#                 # and Cancel all pending orders
#                 for o in self.p.orders:
#                     order = self.p.orders[o]
#                     if order.alive :
#                         self.cancel(order)
#                 self.p.orders = dict()
                
#                 if len(self) == self.size:
#                      self.p.data.loc[self.p.data['exit_time'].isna() ,'exit_time'] = self.datas[0].datetime.datetime(0)
                   

### Data Feeds

In [ ]:
class glove_1M_full_dates_hour_rolling_mean_feed(btfeeds.GenericCSVData):
    lines = ( 'sentiment_mean','sentiment_weighted_mean', 'rolling_mean', 'rolling_mean_weighted',)
    params = (
        ('nullvalue', -2),
        ('dtformat', '%Y-%m-%d %H:%M:%S'),
        ('datetime', 0),
        ('open', 1),
        ('close', 2),
        ('low', 4),
        ('high', 3),
        ('volume', 5),
        ('sentiment_mean', 6),
        ('sentiment_weighted_mean', 7),
        ('rolling_mean', 10),
        ('rolling_mean_weighted', 11),
        
        ('time', -1),
        ('openinterest', -1),
    )

In [ ]:
class glove_final_feed_day(btfeeds.GenericCSVData):
    lines = ( 'sentiment_mean', 'rounded_sum', 'strong_rounded_sum',)
    params = (
        ('nullvalue', -2),
        ('dtformat', '%Y-%m-%d'),
        ('datetime', 0),
        
        ('sentiment_mean', 1),
        ('rounded_sum', 2),
        ('strong_rounded_sum', 3),
        
        ('high', 4),
        ('low', 5),
        ('open', 6),
        ('close', 7),
        ('volume', 8),
       
        ('time', -1),

        ('openinterest', -1),
    )

#### DAY

In [ ]:
class glove_final_feed_day_ma(btfeeds.GenericCSVData):
    lines = ( 'sentiment_mean', 'rounded_sum', 'strong_rounded_sum','ma',)
    params = (
        ('nullvalue', -2),
        ('dtformat', '%Y-%m-%d'),
        ('datetime', 0),
        
        ('sentiment_mean', 1),
        ('rounded_sum', 2),
        ('strong_rounded_sum', 3),
        
        ('high', 4),
        ('low', 5),
        ('open', 6),
        ('close', 7),
        ('volume', 8),
        
        ('ma', 10),
        ('time', -1),
        ('openinterest', -1),
    )

In [ ]:
class glove_final_feed_hour(btfeeds.GenericCSVData):
    lines = ( 'sentiment_mean', 'rounded_sum', 'strong_rounded_sum',)
    params = (
        ('nullvalue', -2),
        ('dtformat', '%Y-%m-%d %H'),
        ('datetime', 0),
        
        ('sentiment_mean', 1),
        ('rounded_sum', 2),
        ('strong_rounded_sum', 3),
        
        ('high', 4),
        ('low', 5),
        ('open', 6),
        ('close', 7),
        ('volume', 8),
       
        ('time', -1),
        ('openinterest', -1),
    )

#### HOUR

In [ ]:
class glove_final_feed_hour_ma(btfeeds.GenericCSVData):
    lines = ( 'sentiment_mean', 'rounded_sum', 'strong_rounded_sum','ma',)
    params = (
        ('nullvalue', -2),
        ('dtformat', '%Y-%m-%d %H'),
        ('datetime', 0),
        
        ('sentiment_mean', 1),
        ('rounded_sum', 2),
        ('strong_rounded_sum', 3),
        
        ('high', 4),
        ('low', 5),
        ('open', 6),
        ('close', 7),
        ('volume', 8),
        
        ('ma', 16),
        ('time', -1),
        ('openinterest', -1),
    )

### Cerebro

In [ ]:
def rd(x):
    return round(x,2)
thresholds = np.linspace(0.91,1.03,13)
thresholds = list(map(rd,list(thresholds)))
max_positions = range(1,15)

In [ ]:
import pathlib
import backtrader.analyzers as btanalyzers
    # Create a cerebro entity
val_list_s = []
for s in max_positions:
    val_list2_s = []
    for t in thresholds:
        cerebro = bt.Cerebro(cheat_on_open=True)

        # Add a strategy
        #strat = Random1
        # strat = TestStrategy
        # strat = BuyAndHold
        # strat = HighMidLowTryX2
        # strat = HighMidLowTryX
        #strat = HighMidLowTryXma
        strat = HighMidLowTryXmamul
        # strat = HighMidLowBest
        # strat = BuyHold
        # strat = OverUnderMean
        cerebro.addstrategy(strat, th=t, p=s)
        # strat = BuyAndHold
        # cerebro.addstrategy(strat)

        #data = glove_100k_91_min_feed(dataname = '../Data Preprocessed/glove_100k_91_min.csv', timeframe = bt.TimeFrame.Minutes, compression = 1)
        # data = glove_1M_full_dates_min_feed(dataname = '../Data Preprocessed/glove_1M_full_dates_min.csv', timeframe = bt.TimeFrame.Minutes, compression = 1)
        #data = glove_1M_full_dates_hour_feed(dataname = '../Data Preprocessed/glove_1M_full_dates_hour.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
        #data = final_dataset_feed(dataname = '../final_dataset3.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
        # data = glove_1M_full_dates_hour_rolling_mean_feed(dataname = './glove_1M_full_dates_hour_rolling_mean.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
        # data = glove_final_feed_day(dataname = '../Data Preprocessed/bert_day_fin.csv', timeframe = bt.TimeFrame.Days, compression = 1)
        
        #data = glove_final_feed_day_ma(dataname = '../Data Preprocessed/bert_day_user_fin.csv', timeframe = bt.TimeFrame.Days, compression = 1)
        data = glove_final_feed_hour_ma(dataname = '../Data Preprocessed/bert_hour_user_fin.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)

        # data = glove_final_feed_day(dataname = '../Data Preprocessed/glove_day_fin.csv', timeframe = bt.TimeFrame.Days, compression = 1)
        # data = glove_final_feed_hour(dataname = '../Data Preprocessed/bert_hour_fin.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
        # data = glove_final_feed_hour(dataname = '../Data Preprocessed/bert_hour_user_fin.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)

        cerebro.broker.setcommission(commission=0, margin =0)
        cerebro.broker.set_coc(True)
        # Add the Data Feed to Cerebro
        cerebro.adddata(data)

        #sizers
        # cerebro.addsizer(bt.sizers.AllInSizer)
        # cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
        #cerebro.addsizer(bt.sizers.PercentSizer, percents = 95/s)
        # cerebro.addsizer(bt.sizers.PercentSizer, percents = 0.01)
        # cerebro.addsizer(bt.sizers.SizerFix, stake=0.1)

        #observers
        cerebro.addobserver(bt.observers.BuySell)

        #analyzers

        # cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe', timeframe=bt.TimeFrame.Years,compression=1)
        # cerebro.addanalyzer(btanalyzers.SQN, _name='sqn')

        #cerebro.add_order_history(orders, notify=True)

        # Set our desired cash start
        cerebro.broker.setcash(1)


        # Print out the starting conditions
        print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Run over everything
        cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

        results = cerebro.run()

        strat = results[0]
        pyfoliozer = strat.analyzers.getbyname('pyfolio')

        returns_high, positions_high, transactions_high, gross_lev_high = pyfoliozer.get_pf_items()

        print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        val = results[0].params.val.reset_index(drop = True)
        val.index = val["datetime"]
        val_list2_s.append(val["value"].astype(float))
    val_list_s.append(val_list2_s)

In [ ]:
import ffn

In [ ]:
import math
x = np.array((list(map(calmar2,val_list_s))))
fig, ax =  plt.subplots(figsize=(15, 6))
sns.heatmap(x, cmap='hot')
ax.set_xticks(np.arange(len(thresholds)))
ax.set_yticks(np.arange(len(max_positions)))
ax.set_xticklabels(thresholds)
ax.set_yticklabels(max_positions)
fig.tight_layout()
plt.show()

In [ ]:
val_list_s[5][8].calc_stats().display()

In [ ]:
import math
x = np.array((list(map(calmar2,val_list_s))))
fig, ax =  plt.subplots(figsize=(15, 6))
sns.heatmap(x, cmap='hot')
ax.set_xticks(np.arange(len(thresholds)))
ax.set_yticks(np.arange(len(max_positions)))
ax.set_xticklabels(thresholds)
ax.set_yticklabels(max_positions)
fig.tight_layout()
plt.show()

In [ ]:
import math
x = np.array((list(map(dd2,val_list_s))))
fig, ax =  plt.subplots(figsize=(15, 6))
sns.heatmap(x, cmap='hot')
ax.set_xticks(np.arange(len(list(thresholds))))
ax.set_yticks(np.arange(len(max_positions)))
ax.set_xticklabels(list(thresholds))
ax.set_yticklabels(max_positions)
fig.tight_layout()
plt.show()

In [ ]:
import math
x = np.array((list(map(dd2,val_list))))
fig, ax =  plt.subplots(figsize=(15, 6))
sns.heatmap(x, cmap='hot')
ax.set_xticks(np.arange(len(list(thresholds))))
ax.set_yticks(np.arange(len(max_positions)))
ax.set_xticklabels(list(thresholds))
ax.set_yticklabels(max_positions)
fig.tight_layout()
plt.show()

In [ ]:
plt.plot(val_list_s[-1][-1])

In [ ]:
def calmar(df):
    return df.calc_calmar_ratio()
def calmar2(l_df):
    return list(map(calmar,l_df))

In [ ]:
def dd(df):
    return df.calc_max_drawdown()
def dd2(l_df):
    return list(map(dd,l_df))

In [ ]:
for v in val_list2:
    print(v.calc_calmar_ratio())

In [ ]:
# df = results[0].params.data
# df.to_csv('low.csv')
# df

In [ ]:
# print('Sharpe Ratio:', results[0].analyzers.mysharpe.get_analysis())
# print('SQN:', results[0].analyzers.sqn.get_analysis())

In [ ]:
val = results[0].params.val.reset_index(drop = True)
val

In [ ]:
val.index = val["datetime"]

In [ ]:
val["value"].astype(float).calc_stats().display()

In [ ]:
plt.plot(val.value)

In [ ]:
val['max'] = 0
for i,r in val.iterrows():
    val.loc[i,'max'] = val.value.values[:i+1].max()
val

In [ ]:
val["dd"] = (val["max"] - val["value"])/val["max"]
val

In [ ]:
val['day'] = val.datetime.apply(lambda x: str(x)[:10])
val['ret'] = val.value.pct_change().fillna(0)
ret_daily = val.groupby(['day']).first().value.pct_change().fillna(0)
ret_daily

In [ ]:
ret_daily.to_csv('low.csv')

In [ ]:
val

In [ ]:
def sharpe_ratio(return_series, N, rf):
#     print(return_series)
    mean = return_series.mean() * N -rf
    sigma = return_series.std() * np.sqrt(N)
    return mean / sigma

N = 365 #255 trading days in a year
rf =0.01 #1% risk free rate
sharpes = pd.DataFrame(ret_daily).apply(sharpe_ratio, args=(N,rf,), axis = 0)


In [ ]:
sharpes

In [ ]:
def max_drawdown(return_series):
    comp_ret = (return_series+1).cumprod()
    peak = comp_ret.expanding(min_periods=1).max()
    dd = (comp_ret/peak)-1
    return dd.min()


max_drawdowns =  pd.DataFrame(ret_daily).apply(max_drawdown,axis=0)

In [ ]:
max_drawdowns

In [ ]:
loss_hour = val['ret']*-1
VaR_hour99 = np.quantile(loss_hour,0.99)
VaR_hour95 = np.quantile(loss_hour,0.95)
VaR_hour99, VaR_hour95

In [ ]:
ES_hour99  = loss_hour[loss_hour > VaR_hour99].mean()
ES_hour95  = loss_hour[loss_hour > VaR_hour95].mean()
ES_hour99, ES_hour95

In [ ]:
loss_daily = ret_daily*-1
VaR_day99 = np.quantile(loss_daily.values,0.99)
VaR_day95 = np.quantile(loss_daily.values,0.95)
VaR_day99, VaR_day95

In [ ]:
ES_day99  = loss_daily[loss_daily > VaR_day99].mean()
ES_day95  = loss_daily[loss_daily > VaR_day95].mean()
ES_day99, ES_day95

In [ ]:
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.io import curdoc

from bokeh.io import output_notebook
output_notebook()

In [ ]:
p2 = figure(x_axis_type="datetime", title="Equity Curve", plot_width=800, plot_height=600)

p2.line(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val['value'].values, legend_label='GloVe - Average Hour Sentiment', color='steelblue')

p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
p2 = figure(x_axis_type="datetime", title="Equity Curve", plot_width=800, plot_height=600)
#p2.title_text_font_size='40pt'


p2.line(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val['value'].values, legend_label='GloVe - Average Hour Sentiment', color='steelblue')
p2.line(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val['max'].values, legend_label='High-Water Mark', color='darkgoldenrod')
p2.varea(y1 = val['dd'].values, y2 = val['dd'].values*0, x =pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, legend_label='Relative Drawdown', color='firebrick')
p2.line(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val['dd'].max(), legend_label='Maximum Drawdown', line_dash = "dashed", color='black')

p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")
p2.legend.label_text_font_size = '12pt'
bokeh.plotting.show(p2)

In [ ]:
p2 = figure(x_axis_type="datetime", title="Returns (Hourly)", plot_width=800, plot_height=600)

p2.vbar(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, top =val['ret'].values, width = .9, legend_label='GloVe - Average Hour Sentiment', fill_color ='steelblue')

p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
p2 = figure(x_axis_type="datetime", title="Returns (Daily)", plot_width=800, plot_height=600)

p2.vbar(list(set(pd.to_datetime(val['day'],format = '%Y-%m-%d %H:%M:%S').values)), top =ret_daily.values, width = .9, legend_label='GloVe - Average Hour Sentiment', fill_color ='steelblue')

p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
!pip install yfinance
#Compare with S&P 500
import yfinance as yf

In [ ]:
spy_ohlc_df = yf.download('SPY', start=val.iloc[0]["day"], end=val.iloc[-1]["day"])
spy_ohlc_df["Open"] = spy_ohlc_df["Open"]/spy_ohlc_df.iloc[0]["Open"]
spy_ohlc_df

In [ ]:
val_daily = val.groupby(['day']).value.first()
val_daily

In [ ]:
p2 = figure(x_axis_type="datetime", title="Equity Curve", plot_width=800, plot_height=600)

p2.line(pd.to_datetime(val_daily.index,format = '%Y-%m-%d').values, val_daily.values, legend_label='GloVe - Average Hour Sentiment Strategy', color='steelblue')
p2.line(pd.to_datetime(spy_ohlc_df.index,format = '%Y-%m-%d').values, spy_ohlc_df["Open"].values, legend_label='S&P 500', color='firebrick')

p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
#compare with btc
p2 = figure(x_axis_type="datetime", title="Trading History", plot_width=800, plot_height=600)


p2.line(pd.to_datetime(btc['hour'],format = '%Y-%m-%d %H:%M:%S').values, btc['Close'].values/btc.iloc[0]['Close'], legend_label='BTC price', color='firebrick')
p2.line(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val['value'].values, legend_label='GloVe - Average Hour Sentiment', color='steelblue')

p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
df2 = results[0].params.data
df2

In [ ]:
df = results[0].params.data
df.to_csv('low.csv')
df

In [ ]:
plt.plot(np.abs(df['weight'].values))

In [ ]:
df['profits'] = df['weight']*(df['exit_price']-df['entry_price'])/df['entry_price']
df[df['position']==1].profits.sum(),df[df['position']==-1].profits.sum()

In [ ]:
val1 = val[val['datetime'].dt.year == 2017]
p2 = figure(x_axis_type="datetime", title="Equity Curve 2017", plot_width=800, plot_height=600)

p2.line(pd.to_datetime(val1['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val1['value'].values, legend_label='GloVe - Average Hour Sentiment Strategy', color='steelblue')

p2.legend.location = "top_left"

#output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)


In [ ]:
val1 = val[val['datetime'].dt.year == 2018]
val1['value'] = val1['value']/val1.iloc[0]['value']
p2 = figure(x_axis_type="datetime", title="Equity Curve 2018", plot_width=800, plot_height=600)

p2.line(pd.to_datetime(val1['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val1['value'].values, legend_label='GloVe - Average Hour Sentiment Strategy', color='steelblue')

p2.legend.location = "top_left"

output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
p2 = figure(x_axis_type="datetime", title="Asset Curve", plot_width=800, plot_height=600)

p2.line(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val['value'].values, legend_label='BTC price', color='steelblue')
p2.line(pd.to_datetime(val['datetime'],format = '%Y-%m-%d %H:%M:%S').values, val['Open'].values, legend_label='BTC price', color='steelblue')

p2.legend.location = "top_left"

bokeh.plotting.show(p2)

In [ ]:
df

In [ ]:
btc = pd.read_csv('../Data Preprocessed/bert_day_fin.csv')
btc

In [ ]:
btc = pd.read_csv('./glove_1M_full_dates_hour_rolling_mean.csv')
btc

In [ ]:
p2 = figure(x_axis_type="datetime", title="Trading History", plot_width=800, plot_height=600)

p2.circle(df[df['position']==1]['entry_time'],df[df['position']==1]['entry_price'],color='green', legend_label='Long position')
p2.circle(df[df['position']==-1]['entry_time'],df[df['position']==-1]['entry_price'],color='red',alpha=0.6,legend_label='Short position')
p2.circle(df['exit_time'],df['exit_price'],color='yellow',alpha=0.1,legend_label='Close position')
p2.line(pd.to_datetime(btc['hour'],format = '%Y-%m-%d %H:%M:%S').values, btc['Close'].values, legend_label='BTC price', color='steelblue')

p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
def plot2(self, my_data, color='steelblue', line=2):
    df = self.metrics.dropna()
    df['sign'] = df['position']
    df['change'] = df['Position USD'].diff().fillna(1)
    long = df[df['sign'] == 1][df['change'] != 0]
    short = df[df['sign'] == -1][df['change'] != 0]
    close = df[df['sign'] == 0][df['change'] != 0]
    p2 = figure(x_axis_type="datetime", title="Trading History", plot_width=800, plot_height=600)
    # p2.grid.grid_line_alpha = 1
    p2.xaxis.axis_label = 'Date'
    p2.yaxis.axis_label = 'Price'
    # p2.ygrid.band_fill_color = "lightsteelblue"
    # p2.ygrid.band_fill_alpha = 0.1

    curdoc().theme = 'dark_minimal'
    p2.circle(long.index.values, long['Price USD'].values, size=4, legend_label='long',
              color='mediumseagreen', alpha=1)
    p2.circle(short.index.values, short['Price USD'].values, size=4, legend_label='short',
              color='indianred', alpha=1)
    p2.circle(close.index.values, close['Price USD'].values, size=4, legend_label='close',
              color='yellow', alpha=1)
    # p2.line(self.data.index.values, self.data['Open'].values, legend_label='avg', color='navy')
    p2.line(my_data.index.values, my_data['Open'].values, legend_label='BTC price', color='grey', alpha=0.2)
    p2.line(self.index_plot, self.data_plot, legend_label='Trading section', color=color, alpha=0.8,
            line_width=line)
    p2.legend.location = "top_left"

    # output_file("stocks.html", title="stocks.py example")

    bokeh.plotting.show(p2)

In [ ]:
df = results[0].params.data
df

In [ ]:
def weekday_match(x):
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    return days[x]

In [ ]:
from datetime import datetime, timedelta
df['entry_time'] = df['entry_time'].apply(lambda x: x-timedelta(hours=1))
df['weekday']= df['entry_time'].apply(lambda x: x.weekday())
df['hour']= df['entry_time'].apply(lambda x: str(x)[11:])
df['weekday'] = df['weekday'].apply(lambda x: weekday_match(x))
df

In [ ]:
heatm = df.groupby(['weekday','hour']).position.apply(np.mean)
heatm

In [ ]:
#Heatmap 1 : Average position size per hour per weekday
#Heatmap 2 : Average direction (long/short) per hour per weekday
days = sorted(list(set(df.weekday.values)))
hours = sorted(list(set(df.hour.values)))
# heatm = pd.DataFrame(columns=sorted(hours), index = sorted(days))
# heatm2 = pd.DataFrame(columns=sorted(hours), index = sorted(days))

In [ ]:
import seaborn as sns

cmap = sns.diverging_palette(10, 110, s=75, l=50, n=9,center = 'dark', as_cmap=True)

fig = plt.figure(figsize=(15, 5))
sns.heatmap(heatm.unstack('hour'),cmap = cmap, center = 0)

In [ ]:

heatm = heatm.fillna(0.)
heatm2 = heatm2.fillna(0.)
algo.metrics = algo.metrics.fillna(0.)
count = 0
for idx, row in df.iterrows():
    d = row['day']
    h = row['hour']
    if row['Position USD'] !=0:
        count += 1
        heatm.loc[d][h]  += row['Position USD']
        heatm2.loc[d][h] = np.sign(heatm.loc[d][h])
heatm = heatm/count
heatm.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
heatm2.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [ ]:
#df['profit'] = 
df['profit'] = (df['exit_price']-df['entry_price'])/df['entry_price']*df['position']
df

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
np.sum(df['profit'].fillna(0).values)

In [ ]:
df.to_csv('./Results/glove_1M_full_dates_hour_rolling_mean_low')

In [ ]:
df1 = pd.read_csv('./Results/glove_1M_full_dates_hour_rolling_mean_low', index_col = 0)
df1

### Backtest Results Analysis

In [ ]:
time()

In [ ]:
#Random
from timeit import default_timer as timer  
values = []
start = timer()
for i in range(100):
    
    print(timer() - start, " sec : Iterration", i)
    start = timer()
    # Create a cerebro entity
    cerebro = bt.Cerebro(cheat_on_open=True)

    # Add a strategy
    strat = Random1
    # strat = TestStrategy
    # strat = BuyAndHold
    #strat = HighMidLow
    # strat = BuyHold
    # strat = OverUnderMean
    cerebro.addstrategy(strat)
    # strat = BuyAndHold
    # cerebro.addstrategy(strat)

    #data = glove_100k_91_min_feed(dataname = '../Data Preprocessed/glove_100k_91_min.csv', timeframe = bt.TimeFrame.Minutes, compression = 1)
    # data = glove_1M_full_dates_min_feed(dataname = '../Data Preprocessed/glove_1M_full_dates_min.csv', timeframe = bt.TimeFrame.Minutes, compression = 1)
    #data = glove_1M_full_dates_hour_feed(dataname = '../Data Preprocessed/glove_1M_full_dates_hour.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
    #data = final_dataset_feed(dataname = '../final_dataset3.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
    data = glove_1M_full_dates_hour_rolling_mean_feed(dataname = './glove_1M_full_dates_hour_rolling_mean.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
    #data = glove_final_feed_day(dataname = '../Data Preprocessed/glove_hour_fin.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
    cerebro.broker.setcommission(commission=0.0)
    cerebro.broker.set_coc(True)
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    #sizers
    # cerebro.addsizer(bt.sizers.AllInSizer)
    # cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
    cerebro.addsizer(bt.sizers.PercentSizer, percents = 95)
    # cerebro.addsizer(bt.sizers.PercentSizer, percents = 0.01)
    # cerebro.addsizer(bt.sizers.SizerFix, stake=0.1)

    #observers
    cerebro.addobserver(bt.observers.BuySell)

    #analyzers

#     cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe', timeframe=bt.TimeFrame.Years,compression=1)
#     cerebro.addanalyzer(btanalyzers.SQN, _name='sqn')

    #cerebro.add_order_history(orders, notify=True)

    # Set our desired cash start
    cerebro.broker.setcash(1)


    # Print out the starting conditions
#     print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

#     # Run over everything
#     cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    results = cerebro.run()

    
    values.append(results[0].params.val)

In [ ]:
b1 = pd.read_csv('../Data Preprocessed/bert_hour_fin.csv',index_col=0)
b1

In [ ]:
np.random.randint(-1,2,(b1.shape[0],1000))

In [ ]:
v_rand = np.multiply(b1.Open.pct_change().fillna(0).values,np.random.randint(-1,2,(b1.shape[0],1000)).T)

In [ ]:
v_rand = pd.DataFrame(v_rand.T)
rets = v_rand.copy()
rets

In [ ]:
v_rand.iloc[0] = 1
v_rand

In [ ]:

for i,r in v_rand.iterrows():
    if i>0:
        v_rand.iloc[i] =  v_rand.iloc[i-1]*(v_rand.iloc[i].T + 1)


In [ ]:
rand = pd.DataFrame(v_rand)
rand = pd.concat([rand,pd.DataFrame(b1.index)], axis =1)
rand['day'] = rand["hour"].apply(lambda x: str(x)[:10])

In [ ]:
rets["day"] = rand["day"]
rets_daily = rets.groupby(['day']).first()
rets_daily.apply(sharpe_ratio, args=(N,rf,), axis=0).describe()

In [ ]:
rets_daily.sum(axis=0).describe()

In [ ]:
plt.plot(v_rand.values)
plt.yscale("log")

In [ ]:
values2 = []
for v in values:
    values2.append(v['value'])
v_all = pd.concat(values2, axis=1)
avg = v_all.apply(np.mean, axis=1)

In [ ]:
from scipy import stats

In [ ]:
round(94/10)

In [ ]:
pd.concat([v_all,v['datetime']],axis =1 ).to_csv('Random_strategies.csv')

In [ ]:
rand = pd.read_csv('Random_strategies.csv', index_col = 0)
rand

In [ ]:
rand['day'] = rand.datetime.apply(lambda x: str(x)[:10])
rets = rand.groupby(['day']).first().iloc[:,:-1].astype(float).pct_change(axis=0).fillna(0)

rets.apply(sharpe_ratio, args=(N,rf,), axis=0)


In [ ]:
from scipy import stats
stats.percentileofscore(rets.apply(sharpe_ratio, args=(N,rf,), axis=0).values, 1.88)

In [ ]:
import itertools  
from bokeh.palettes import Category20_20 as palette
p2 = figure(x_axis_type="datetime", title="Random Strategies", plot_width=1200, plot_height=600,y_axis_type="log")
#colors = itertools.cycle(palette) 
colors = ['silver',
          'pink',
          'yellow',
          'orange',
          'green',
          'blue',
          'purple',
          'brown',
          'red',
          'black',
          'darkgoldenrod'
         ]
labels = ['0-5%',
         '5-15%',
         '15-25%',
         '25-35%',
         '35-45%',
         '45-55%',
         '55-65%',
         '65-75%',
         '75-85%',
         '85-95%',
         '95-100%',]
for v in values:
    prc = round(stats.percentileofscore(v_all.iloc[-1].values, v.iloc[-1]['value'])/10)
    color = colors[prc]
    label = labels[prc]
    p2.line(v['datetime'], v['value'], color = color, alpha = 0.3,  legend_label = label)
    
p2.xaxis.axis_label_text_font_size = "100pt"
p2.yaxis.axis_label_text_font_size = "100pt"
p2.line(val['datetime'].values, val['value'].values, width = 3, color = 'darkred', legend_label = "Main Strategy")
p2.legend.location = "top_left"

# output_file("stocks.html", title="stocks.py example")

bokeh.plotting.show(p2)

In [ ]:
#Trading History
cerebro.plot()

In [ ]:
ret_low = returns_fin2.values
ret_mid = returns_fin.values

In [ ]:
#Daily Returns
from matplotlib import pyplot as plt
returns2 = pd.DataFrame(returns_high.values, columns = ['returns'])
returns2.index = returns.index
returns2['datetime'] = returns.index
returns_fin22 = returns2.groupby(['datetime'])['returns'].apply(np.sum)
# plt.title("Daily Returns")
# plt.bar(returns.index, returns_fin)
# plt.hlines(0,returns_fin.index[0],returns_fin.index[-1],color = "black")
# plt.show()

In [ ]:
ret_high = returns_fin22.values

In [ ]:
np.std(ret_high)

In [ ]:
import numpy as np, statsmodels.stats.api as sms

In [ ]:
sms.CompareMeans(ret_high, ret_mid).tconfint_diff(usevar='unequal')

In [ ]:
transactions

In [ ]:
#Transaction Returns
plt.title("Transaction Returns")
plt.bar(transactions.index,transactions['value'])
plt.hlines(0,transactions.index[0],transactions.index[-1],color = "black")
plt.show()

In [ ]:
# Sharpe Ratio
print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis())

#### High Mid Low Analysis

In [ ]:
# Low
cerebro.plot()

In [ ]:
# Mid
cerebro.plot()

In [ ]:
#high
cerebro.plot()

In [ ]:
import pathlib
import backtrader.analyzers as btanalyzers
# Create a cerebro entity

cerebro = bt.Cerebro(cheat_on_open=True)

# Add a strategy
#strat = Random1
# strat = TestStrategy
# strat = BuyAndHold
# strat = HighMidLowTryX2
# strat = HighMidLowTryX
#strat = HighMidLowTryXma
strat = HighMidLowTryXmamulh
# strat = HighMidLowBest
# strat = BuyHold
# strat = OverUnderMean
cerebro.addstrategy(strat, th=1, p=3)
# strat = BuyAndHold
# cerebro.addstrategy(strat)

#data = glove_100k_91_min_feed(dataname = '../Data Preprocessed/glove_100k_91_min.csv', timeframe = bt.TimeFrame.Minutes, compression = 1)
# data = glove_1M_full_dates_min_feed(dataname = '../Data Preprocessed/glove_1M_full_dates_min.csv', timeframe = bt.TimeFrame.Minutes, compression = 1)
#data = glove_1M_full_dates_hour_feed(dataname = '../Data Preprocessed/glove_1M_full_dates_hour.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
#data = final_dataset_feed(dataname = '../final_dataset3.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
# data = glove_1M_full_dates_hour_rolling_mean_feed(dataname = './glove_1M_full_dates_hour_rolling_mean.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
# data = glove_final_feed_day(dataname = '../Data Preprocessed/bert_day_fin.csv', timeframe = bt.TimeFrame.Days, compression = 1)
# data = glove_final_feed_day_ma(dataname = '../Data Preprocessed/bert_day_user_fin.csv', timeframe = bt.TimeFrame.Days, compression = 1)

# data = glove_final_feed_day(dataname = '../Data Preprocessed/glove_day_fin.csv', timeframe = bt.TimeFrame.Days, compression = 1)
# data = glove_final_feed_hour(dataname = '../Data Preprocessed/bert_hour_fin.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)
data = glove_final_feed_hour_ma(dataname = '../Data Preprocessed/bert_hour_user_fin.csv', timeframe = bt.TimeFrame.Minutes, compression = 60)

cerebro.broker.setcommission(commission=0, margin =0)
cerebro.broker.set_coc(True)
# Add the Data Feed to Cerebro
cerebro.adddata(data)

#sizers
# cerebro.addsizer(bt.sizers.AllInSizer)
# cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
#cerebro.addsizer(bt.sizers.PercentSizer, percents = 95/s)
# cerebro.addsizer(bt.sizers.PercentSizer, percents = 0.01)
# cerebro.addsizer(bt.sizers.SizerFix, stake=0.1)

#observers
cerebro.addobserver(bt.observers.BuySell)

#analyzers

# cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe', timeframe=bt.TimeFrame.Years,compression=1)
# cerebro.addanalyzer(btanalyzers.SQN, _name='sqn')

#cerebro.add_order_history(orders, notify=True)

# Set our desired cash start
cerebro.broker.setcash(1)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

results = cerebro.run()

strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')

returns_high, positions_high, transactions_high, gross_lev_high = pyfoliozer.get_pf_items()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
val = results[0].params.val.reset_index(drop = True)
val.index = val["datetime"]
# val_list2_s.append(val["value"].astype(float))
# val_list_s.append(val_list2_s)

In [ ]:
val["value"].astype(float).calc_stats().display()

In [ ]:
plt.plot(val.value)